In [ ]:
import pandas as pd
file_path = './data/processed_full_recipes_from_2008.csv'
df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')
df = df.dropna()

def create_rag_txt_file(data, path_to_save):
    rows = []
    for _, row in data.iterrows():
        res_str = ""
        res_str += "Name: " + row['name'] + " Tags: " + row['tags'] + " Cooking steps: " + row['steps_processed'].replace("\n", " ")
        rows.append(res_str) 
    
    text = "\n".join(rows)
    f = open(path_to_save, 'w')
    f.write(text)
    f.close()
    print(f"File saved! File lenght : {len(text)}")
    return text

text = create_rag_txt_file(df, "./data/recipes.txt")

File saved! File lenght : 68014778


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from openai import OpenAI
import os
import streamlit as st

In [4]:
from langchain_community.document_loaders.text import TextLoader
txt_file_path = "./data/recipes.txt"
loader = TextLoader(file_path=txt_file_path, encoding="utf-8", )
data = loader.load()

In [5]:
from langchain_text_splitters.character import CharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=200)
data = text_splitter.split_documents(data)
print(len(data))

44085


In [7]:
embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))

C:\Users\ykuzm\AppData\Local\Temp\ipykernel_13168\3950337156.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))


In [8]:
from tqdm import tqdm

db = None
with tqdm(total=len(data), desc="Ingesting documents") as pbar:
    for d in data:
        if db:
            db.add_documents([d])
        else:
            db = FAISS.from_documents([d], embeddings)
        pbar.update(1) 

Ingesting documents:   0%|          | 0/44085 [00:00<?, ?it/s]

Ingesting documents:   2%|▏         | 738/44085 [04:23<4:18:20,  2.80it/s] 


KeyboardInterrupt: 

In [38]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [39]:
retriever = new_db.as_retriever()

In [63]:
llm = ChatOpenAI(api_key = os.getenv('OPENAI_API_KEY'), temperature=0.7, model_name="gpt-3.5-turbo-0125")

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user input "
    "which might reference context in the chat history, "
    "formulate a standalone input which can be understood "
    "without the chat history. Do NOT answer the user input, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [64]:
type(history_aware_retriever)

langchain_core.runnables.base.RunnableBinding

In [54]:
### Answer question ###
system_prompt = (
    "You are a helpful and creative recipe generator."
    "User give you their products and their preferences. You can add product to complete the recipe."
    "If you don't know the recipe of the dish, don't suggest it."
    "You can ONLY give cooking related advices, if user ask's something that is not related to cooking: Sorry, i can't reply to that."
    "Use the following pieces of retrieved context to answer "
    "the user prompt."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [55]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [65]:
type(question_answer_chain)

langchain_core.runnables.base.RunnableBinding

In [59]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [60]:
conversational_rag_chain.invoke(
    {"input": "Can you create cooking recipes?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Yes, I can help you create cooking recipes based on specific ingredients and preferences you provide. Just let me know what you have in mind, and I'll come up with some delicious recipes for you!"

In [61]:
conversational_rag_chain.invoke(
    {"input": "What can i cook with cheese and chicken breasts?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'One tasty dish you can make with cheese and chicken breasts is "Spinach and Cheese Stuffed Chicken Breast." Here\'s a simple recipe for you:\n\nIngredients:\n- Chicken breasts\n- Olive oil\n- Hidden Valley ranch seasoning and dressing mix\n- Spinach\n- Parmesan cheese\n- Mozzarella cheese\n- Bread crumbs\n- Beaten egg\n- Olives (optional)\n\nInstructions:\n1. Butterfly the chicken breasts and drizzle with olive oil.\n2. Sprinkle the chicken breasts with Hidden Valley seasoning and dressing mix, then set aside.\n3. In a sauté pan, cook spinach until it starts to cook down.\n4. Remove spinach from heat and add it to a medium bowl with parmesan cheese, half of the mozzarella cheese, bread crumbs, beaten egg, and olives if using. Mix well.\n5. Place a spoonful of the spinach mixture in the middle of each chicken breast and fold over, covering the spinach completely.\n6. Sprinkle with the remaining mozzarella cheese and place on a baking sheet.\n7. Bake at 350 degrees for 20-25 minutes or 